# 1. 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

# 2. 读取数据集

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))    # 这里我们使用iter构造Python迭代器，并使用next从迭代器中获取第一项

[tensor([[-0.1302,  0.8933],
         [ 0.1327,  1.4475],
         [ 0.5654,  0.1513],
         [-0.3900, -1.1950],
         [ 0.7036,  0.1807],
         [ 0.2214,  0.3736],
         [ 0.9538, -0.3348],
         [-0.9888,  0.9213],
         [-1.2267, -1.8029],
         [-0.4357, -0.2360]]),
 tensor([[ 0.8989],
         [-0.4544],
         [ 4.8364],
         [ 7.4782],
         [ 4.9868],
         [ 3.3898],
         [ 7.2712],
         [-0.9092],
         [ 7.8595],
         [ 4.1088]])]

# 3. 定义模型

Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。

我们的模型只有一层，即单层神经网络。

In [3]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))    # 第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1

# 4. 初始化模型参数

In [4]:
net[0].weight.data.normal_(0, 0.01)     # 通过net[0]选择网络中的第一个图层，使用正态分布替换掉data中的值
net[0].bias.data.fill_(0)

tensor([0.])

# 5. 定义损失函数

In [5]:
loss = nn.MSELoss()        # 返回所有样本损失的平均值

# 6. 定义优化算法

小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。

当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置lr值，这里设置为0.03。

In [6]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

# 7. 训练

In [7]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()    # 将优化器中的梯度清零
        l.backward()           # 反向传播计算梯度
        trainer.step()         # 调用step()函数来进行模型参数更新
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000208
epoch 2, loss 0.000107
epoch 3, loss 0.000106


In [8]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0002, -0.0009])
b的估计误差： tensor([8.1062e-05])
